### Load Pre-Processed Data

In [1]:
# retrieve stored values

%store -r x_train 
%store -r x_test 
%store -r y_train 
%store -r y_test 
%store -r yy 
%store -r le


### Setting up CNN

In [2]:
disable_gpu = False

# To disable gpu if wanted
if disable_gpu:
    import os
    os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn import metrics
import tensorflow as tf

if len(tf.config.list_physical_devices('GPU')):
    print("GPU detected")
else:
    print("CPU detected, this may take a while")

2024-08-19 21:33:36.791161: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


GPU detected


2024-08-19 21:33:38.542883: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-19 21:33:38.630764: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-19 21:33:38.631016: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.


In [3]:
# possible way to free up gpu memory after easier https://www.kaggle.com/c/jigsaw-unintended-bias-in-toxicity-classification/discussion/96876

num_rows = 40
num_columns = 174
num_channels = 1

x_train = x_train.reshape(x_train.shape[0], num_rows, num_columns, num_channels)
x_test = x_test.reshape(x_test.shape[0], num_rows, num_columns, num_channels)

num_labels = yy.shape[1]
filter_size = 2

# Construct model 
model = Sequential()

model.add(Conv2D(filters=16, kernel_size=2, input_shape=(num_rows, num_columns, num_channels), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=32, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=64, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=128, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(GlobalAveragePooling2D())

model.add(Dense(num_labels, activation='softmax')) 

2024-08-19 21:33:41.120843: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-19 21:33:41.121399: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-19 21:33:41.121875: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-19 21:33:41.318151: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-19 21:33:41.318427: I tensorflow/compile

### Compiling Model

In [4]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam') 

In [5]:
# Display model architecture summary 
model.summary()

# Calculate pre-training accuracy 
score = model.evaluate(x_test, y_test, verbose=0)
accuracy = 100*score[1]

print("Pre-training accuracy: %.4f%%" % accuracy)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 39, 173, 16)       80        
                                                                 
 max_pooling2d (MaxPooling2D  (None, 19, 86, 16)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 19, 86, 16)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 18, 85, 32)        2080      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 9, 42, 32)        0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 9, 42, 32)         0

2024-08-19 21:33:49.875330: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8907
2024-08-19 21:33:50.070943: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-08-19 21:33:50.582759: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


Pre-training accuracy: 9.8780%


In [ ]:
from keras.callbacks import ModelCheckpoint 
from datetime import datetime 
#import tqdm.notebook as tqdm
import tqdm
from tqdm.keras import TqdmCallback

num_epochs = 100
num_batch_size = 32

checkpointer = ModelCheckpoint(filepath='models/weights.best.basic_cnn.hdf5', 
                               verbose=0, save_best_only=True)
start = datetime.now()
    
history = model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(x_test, y_test), callbacks=[checkpointer,TqdmCallback(verbose=1)], verbose = 0)

duration = datetime.now() - start
print("Training completed in time: ", duration)

0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

2024-08-19 21:33:55.750695: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2024-08-19 21:33:55.875546: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x88e1860 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-08-19 21:33:55.875594: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA GeForce RTX 4070 Ti, Compute Capability 8.9
2024-08-19 21:33:55.913584: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-08-19 21:33:56.135640: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-08-19 21:33:56.208275: I ./tensorflow/compiler/jit/device

### Results from training


In [10]:
# Evaluating the model on the training and testing set
score = model.evaluate(x_train, y_train, verbose=0)
print("Training Accuracy: ", score[1])

score = model.evaluate(x_test, y_test, verbose=0)
print("Testing Accuracy: ", score[1])

Training Accuracy:  0.10230331867933273
Testing Accuracy:  0.09808612614870071


### Save model for use elsewhere

In [14]:
import pickle

# Used to save in the old h5 model, leaving the endoing will save it as a tensorflow model
model.save('sound_category_model')

# sound_model_file = "sound_model.pkl"

# with open(sound_model_file, 'wb') as file:
#     pickle.dump(model, file)

label_encoder_file = "labelencoder.pkl"


    
with open(label_encoder_file, 'wb') as file:
    pickle.dump(le, file)

#%store model


2023-07-01 23:55:54.764849: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,19,86,16]
	 [[{{node inputs}}]]
2023-07-01 23:55:54.773406: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,9,42,32]
	 [[{{node inputs}}]]
2023-07-01 23:55:54.781532: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,4,20,64]
	 [[{{node inputs}}]]
202

INFO:tensorflow:Assets written to: sound_category_model/assets


INFO:tensorflow:Assets written to: sound_category_model/assets
